In [ ]:
from transformers import BertTokenizer, BertForTokenClassification, BertForSequenceClassification
import torch

In [ ]:
# Load the BERT tokenizer and models for token classification and sequence classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
ner_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_ner_labels)
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent_labels)

# Prepare sample data (input_ids and attention_mask) - you'll need to tokenize your actual data
inputs = tokenizer("Your text goes here", return_tensors="pt", padding=True, truncation=True)

# NER forward pass
outputs_ner = ner_model(**inputs)
ner_predictions = torch.argmax(outputs_ner.logits, dim=2)

# Intent classification forward pass
outputs_intent = intent_model(**inputs)
intent_predictions = torch.argmax(outputs_intent.logits, dim=1)

In [ ]:
optimizer = torch.optim.Adam([
    {'params': ner_model.parameters()},
    {'params': intent_model.parameters()}
], lr=1e-5)  # Define optimizer for both models

num_epochs = 3  # Define number of epochs

for epoch in range(num_epochs):
    for batch in data_loader:  # Iterate over your dataset batches
        inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)

        # Forward pass for NER
        outputs_ner = ner_model(**inputs)
        ner_labels = batch['ner_labels']
        ner_loss = torch.nn.CrossEntropyLoss()(outputs_ner.logits.view(-1, num_ner_labels), ner_labels.view(-1))

        # Forward pass for intent classification
        outputs_intent = intent_model(**inputs)
        intent_labels = batch['intent_labels']
        intent_loss = torch.nn.CrossEntropyLoss()(outputs_intent.logits, intent_labels)

        # Total loss (you might adjust weights for different tasks)
        total_loss = ner_loss + intent_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()